In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
pd.set_option('display.precision',4)

Using TensorFlow backend.


In [2]:
#define columns
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis', 'a1', 'a2','a3', 'a4', 'a5', 'a6']

In [3]:
train_df = pd.read_csv('/Users/sohamdeshmukh/Desktop/WISD_HAR_files/WISDM_ar_v1.1_raw.txt', header = None,names = columns)
train_df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user,activity,timestamp,x-axis,y-axis,z-axis,a1,a2,a3,a4,a5,a6
0,33,Jogging,49105962326000,-0.6946,12.6805,0.50395286;,NaN,NaN,NaN,NaN,NaN,NaN
1,33,Jogging,49106062271000,5.0123,11.2640,0.95342433;,NaN,NaN,NaN,NaN,NaN,NaN
2,33,Jogging,49106112167000,4.9033,10.8827,-0.08172209;,NaN,NaN,NaN,NaN,NaN,NaN
3,33,Jogging,49106222305000,-0.6129,18.4964,3.0237172;,NaN,NaN,NaN,NaN,NaN,NaN
4,33,Jogging,49106332290000,-1.1850,12.1085,7.205164;,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train_df.drop(['a1', 'a2','a3', 'a4', 'a5', 'a6'], axis=1, inplace=True)

In [5]:
train_df.head()

,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.6946,12.6805,0.50395286;
1,33,Jogging,49106062271000,5.0123,11.2640,0.95342433;
2,33,Jogging,49106112167000,4.9033,10.8827,-0.08172209;
3,33,Jogging,49106222305000,-0.6129,18.4964,3.0237172;
4,33,Jogging,49106332290000,-1.1850,12.1085,7.205164;


In [6]:
train_df['z-axis'] = train_df['z-axis'].str.replace(';','')
train_df.head()

,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.6946,12.6805,0.50395286
1,33,Jogging,49106062271000,5.0123,11.2640,0.95342433
2,33,Jogging,49106112167000,4.9033,10.8827,-0.08172209
3,33,Jogging,49106222305000,-0.6129,18.4964,3.0237172
4,33,Jogging,49106332290000,-1.1850,12.1085,7.205164


In [7]:
# abalone = pd.read_csv(target_url,header=None, prefix="V")
# abalone.columns = ['Sex', 'Length', 'Diameter', 'Height',
#                    'Whole Wt', 'Shucked Wt',
#                    'Viscera Wt', 'Shell Wt', 'Rings']
# #get summary to use for scaling
# summary = abalone.describe()
# minRings = summary.iloc[3,7]
# maxRings = summary.iloc[7,7]
# nrows = len(abalone.index)

# for i in range(nrows):
#     #plot rows of data as if they were series data
#     dataRow = abalone.iloc[i,1:8]
#     labelColor = (abalone.iloc[i,8] - minRings) / (maxRings - minRings)
#     dataRow.plot(color=plot.cm.RdYlBu(labelColor), alpha=0.5)

# plot.xlabel("Attribute Index")
# plot.ylabel(("Attribute Values"))
# plot.show()

# #renormalize using mean and standard variation, then compress
# # with logit function

# meanRings = summary.iloc[1,7]
# sdRings = summary.iloc[2,7]

# for i in range(nrows):
#     #plot rows of data as if they were series data
#     dataRow = abalone.iloc[i,1:8]
#     normTarget = (abalone.iloc[i,8] - meanRings)/sdRings
#     labelColor = 1.0/(1.0 + exp(-normTarget))
#     dataRow.plot(color=plot.cm.RdYlBu(labelColor), alpha=0.5)

# plot.xlabel("Attribute Index")
# plot.ylabel(("Attribute Values"))
# plot.show()

In [8]:
summary = train_df.describe()
summary

,user,timestamp,x-axis,y-axis
count,1.0982e+06,1.0982e+06,1.0982e+06,1.0982e+06
mean,1.8861e+01,3.3409e+13,6.6287e-01,7.2556e+00
std,1.0214e+01,4.9450e+13,6.8491e+00,6.7462e+00
min,1.0000e+00,0.0000e+00,-1.9610e+01,-1.9610e+01
25%,1.0000e+01,2.0191e+12,-2.8700e+00,3.1700e+00
50%,1.9000e+01,9.7228e+12,2.7000e-01,7.9300e+00
75%,2.8000e+01,4.9966e+13,4.4400e+00,1.1560e+01
max,3.6000e+01,2.0940e+14,1.9950e+01,2.0040e+01


In [9]:
train_df.shape

(1098204, 6)

In [10]:
nrows = len(train_df.index)
nrows

1098204

In [11]:
train_df = train_df[train_df['x-axis'] != 0]
train_df = train_df[train_df['y-axis'] != 0]
train_df = train_df[train_df['z-axis'] != 0]
train_df = train_df[train_df['timestamp'] != 0]
train_df.shape

(1080537, 6)

In [12]:
train_df.dropna(how='any')#to drop if any value in the row has a nan
train_df.shape

(1080537, 6)

In [13]:
train_df.loc[(train_df!=0).any(axis=1)]
train_df.head()

,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.6946,12.6805,0.50395286
1,33,Jogging,49106062271000,5.0123,11.2640,0.95342433
2,33,Jogging,49106112167000,4.9033,10.8827,-0.08172209
3,33,Jogging,49106222305000,-0.6129,18.4964,3.0237172
4,33,Jogging,49106332290000,-1.1850,12.1085,7.205164


In [17]:
label_array = train_df['activity']
label_array.dropna(how = 'any')
label_array = np.array(label_array)
label_array.astype(str)
labels = np.asarray(pd.get_dummies(label_array), dtype = np.float32)
labels

array([[ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.]], dtype=float32)

In [18]:
labels[0]

array([ 0.,  1.,  0.,  0.,  0.,  0.], dtype=float32)

In [22]:
minRings = summary.iloc[3,4]
maxRings = summary.iloc[7,4]

In [25]:
for i in range(nrows):
    #plot rows of data as if they were series data
    dataRow = train_df.iloc[i,1:5]
    labelColor = (train_df.iloc[i,5] - minRings) / (maxRings - minRings)
    dataRow.plot(color=plt.cm.RdYlBu(labelColor), alpha=0.5)

plt.xlabel("Attribute Index")
plt.ylabel(("Attribute Values"))
plt.show()

TypeError: Empty 'DataFrame': no numeric data to plot